# 6 - NBA Player Detail

Now we're going to heavily use the [nba_api](https://github.com/swar/nba_api) data library as it's proven to be a great API for extracting data from https://stats.nba.com.

In [ ]:
# %pip install nba_api

In [ ]:
import utils

In [ ]:
import pandas as pd
import pathlib

In [ ]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players

In [ ]:
BASE_DIR = pathlib.Path().resolve()
SAMPLES_DIR = BASE_DIR / 'samples'
SAMPLE_PLAYERS_DIR = SAMPLES_DIR / 'players'
SAMPLE_PLAYERS_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
salary_df = pd.read_csv('samples/5-player-adj-salaries-audit.csv')
salary_df.columns = [f"{x}".upper() for x in salary_df.columns]

I converted our columns to match the column names in the `nba_api` library.

In [ ]:
random_row = salary_df.sample(n=1)
name = random_row['PLAYER'].item() # .item() will get the value
random_row

Using `.sample(n=1)` will return a random sample of our data. This sample can be as large as you'd like but I chose to return `1` row (`n=1`) to ultimately get `1` player's name for use in the API.

In [ ]:
# name = 'Michael Jordan'
name

In [ ]:
player_results = players.find_players_by_full_name(name) 
player_df = pd.DataFrame(player_results)
player_df.head()

In [ ]:
player_df

In [ ]:
player_id = player_df.iloc[0]['id'].item()
# player_id = player_df.loc[0]['id'].item()

`.iloc` is much like using an index value in a standard python list. `iloc[23]` will yield the 24th element in the DataFrame. `iloc[0]` will return the 1st element at the 0th position. `loc[0]` *may* yield the same result if the index column is not shuffled. 

In [ ]:
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_df = career.get_data_frames()[0]
career_df.head()

In [ ]:
def season_id_to_season_end(val):
    season_start, season_end = val.split("-")
    if f"{season_start}".startswith("1"):
        season_end = f"19{season_end}"
    else:
        season_end = f"20{season_end}"
    return season_end

In [ ]:
career_df['YEAR_START'] = career_df['SEASON_ID'].apply(lambda x: x.split("-")[0])
career_df['YEAR_END'] = career_df['SEASON_ID'].apply(season_id_to_season_end)

In [ ]:
career_df.head()

In [ ]:
year_start = career_df['YEAR_START'].min()
year_end = career_df['YEAR_END'].max()
year_start, year_end

In [ ]:
key_stats = ['GP',
 'GS',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']
    
key_stats_labels = [
    "Games Played",
    "Games Started",
    "Minutes",
    "Field Goals Made",
    "Field Goals Attempted",
    "Field Goal Percentage",
    "3-point Field Goals Made",
    "3-point Field Goals Made Attempted",
    "3-point Field Goal Percentage",
    'Free Throws Made',
    'Free Throws Attempted',
    'Free Throw Percentage',
    'Offensive Rebouns',
    'Defensive Rebouns',
    'Rebouns',
    'Assists',
    'Steals',
    'Blocks',
    'Turnovers',
    'Personal Fouls',
    'Points'
]
   
key_stats_mapping = dict(zip(key_stats, key_stats_labels))
stats_without_perc = [x for x in key_stats if not "pct" in x.lower()]
stats_with_perc = [x for x in key_stats if "pct" in x.lower()]

In [ ]:
career_df

In [ ]:
career_stats_columns = stats_without_perc + ["SEASON_ID"]
career_stats = career_df.copy()[career_stats_columns]
career_stats.head()

In [ ]:
# career_stats = career_stats.astype(int)
career_stats.set_index("SEASON_ID", inplace=True, drop=True)
totals_df = career_stats.rename(columns=key_stats_mapping)
totals = totals_df.sum()

In [ ]:
totals_df

In [ ]:
career_stats_perc = career_df.copy()[stats_with_perc]
averages_df = career_stats_perc.rename(columns=key_stats_mapping)
averages = averages_df.mean()

In [ ]:
totals

In [ ]:
averages

In [ ]:
earnings = salary_df.copy()[salary_df['PLAYER'] == name][['SALARY', 'ADJ_SALARY', 'YEAR_START']]
earnings.set_index("YEAR_START", inplace=True, drop=True)
earnings.tail(n=15)

In [ ]:
total_earnings = earnings.sum().apply(utils.float_to_dollars)
total_earnings

In [ ]:
name

In [ ]:
total_games = career_stats.GP.sum()
total_games

In [ ]:
earnings_per_game = earnings['SALARY'].sum() / total_games
utils.float_to_dollars(earnings_per_game)

In [ ]:
plot = earnings.plot()

### Export Player Data

In [ ]:
PLAYER_DIR = SAMPLE_PLAYERS_DIR / f"{name}"
PLAYER_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
# Eearnings Plot
figure = plot.figure
figure.savefig(PLAYER_DIR / f"salary_over_time.png")

In [ ]:
stats_output = PLAYER_DIR / "stats.xlsx"

In [ ]:
totals_df

In [ ]:
with pd.ExcelWriter(stats_output) as writer:
    totals_df.to_excel(writer, sheet_name='Career Stats')
    totals.to_excel(writer, sheet_name='Career Totals')
    averages.to_excel(writer, sheet_name='Career Averages')
    earnings.to_excel(writer, sheet_name='Yearly Earnings')
    total_earnings.to_excel(writer, sheet_name='Total Earnings')

In [ ]:
name

In [ ]:
import requests

In [ ]:
import requests

url = "https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season=2021-22&SeasonType=Regular%20Season&TeamID=0"
url = "https://stats.nba.com/stats/playercareerstats?LeagueID=&PerMode=Totals&PlayerID=2544"

r = requests.get(url, stream=True)
r.json()